In [1]:
pip install gmaps

In [2]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from configs import g_key

In [3]:
#Load csv in part 1 to a dataframe

#clean city data filepath
path = "../output_data/clean_city_data.csv"

city_df = pd.read_csv(path)
city_df = city_df.dropna()
city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,70.268,52,100,8.187187,PF,1593015836
1,olafsvik,64.89,-23.71,52.808,75,82,9.864889,IS,1593015867
2,carnarvon,-24.87,113.63,57.200,93,0,8.052971,AU,1593015847
3,iqaluit,63.75,-68.51,42.800,87,75,4.697566,CA,1593015852
4,santo augusto,-27.85,-53.78,73.382,60,6,16.955977,BR,1593016069


# Humidity Heatmap

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
#Store Lat and Lng into locatins
locations = city_df[["Lat", "Lng"]]

#Store humidity into humidity
humidity = city_df["Humidity"].astype(float)

In [6]:
#Create humidity Heatmap layer
fig = gmaps.figure()

humid_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                 max_intensity = 100, point_radius = 1)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [7]:
#Narrow down cities to fit weather conditions

#Filter cities with max temps between 60 and 90 degrees
comfortable_df = city_df.loc[(city_df["Max Temp"] > 60) & (city_df["Max Temp"] < 90), :]
#filter by humidity below 80 percent
comfortable_df = comfortable_df.loc[(comfortable_df["Humidity"] < 80), :]
#filter by wind speed below 10 mph
comfortable_df = comfortable_df.loc[(comfortable_df["Wind Speed"] < 10), :]
#filter by cloudiness below 50 %
comfortable_df = comfortable_df.loc[(comfortable_df["Cloudiness"] < 50), :]

#drop rows with na values
comfortable_df = comfortable_df.dropna()

In [9]:
#store into variable 'hotel_df'
hotel_df = comfortable_df
#adds column "Hotel_name"
hotel_df["Hotel Name"] = ""

#finding a hotel within 5000 meters of our most comfortable cities
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row.Lat
    lng = row.Lng
    params = {
        "location": f'{lat}, {lng}',
        "radius": 5000,
        "key" : g_key,
        "type" : "lodging"
    }
    response = requests.get(base_url, params = params).json()
    #updates df with the hotel name
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    #displays message about the issue city and adds "NA" to hold the Hotel name's place
    except:
        print(f'Issues with finding a hotel at {row["City"]}')
        hotel_df.loc[index, "Hotel Name"] = "NA"
        
hotel_df.head()

Issues with finding a hotel at saskylakh
Issues with finding a hotel at yozgat
Issues with finding a hotel at chokurdakh
Issues with finding a hotel at kalengwa
Issues with finding a hotel at ouegoa
Issues with finding a hotel at moshupa
Issues with finding a hotel at tiznit
Issues with finding a hotel at rudnogorsk
Issues with finding a hotel at kayerkan
Issues with finding a hotel at romanovo
Issues with finding a hotel at trapani
Issues with finding a hotel at avera
Issues with finding a hotel at blatnica


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,cape town,-33.93,18.42,60.998,77,0,9.171439,ZA,1593015495,Southern Sun Waterfront Cape Town
22,sabang,5.89,95.32,83.516,72,45,5.547602,ID,1593016070,Hotel Citra
30,karratha,-20.74,116.85,67.838,77,0,1.096099,AU,1593015864,ibis Styles Karratha
35,bubaque,11.28,-15.83,83.120,77,30,9.260916,GW,1593016071,Casa Dora
37,arinos,-15.92,-46.11,82.058,44,1,5.637079,BR,1593016071,Hotel Dias


In [13]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

#Add markers on top of the heatmap
hotel_layer = gmaps.symbol_layer(
    locations, fill_color = "blue",
    stroke_color = 'black', scale = 2,
    info_box_content = hotel_info)

fig.add_layer(hotel_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))